In [ ]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

In [ ]:
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
    
query = "SELECT * FROM pg_catalog.pg_tables where tableowner = 'ajwin';"  # Example query: fetch 10 rows from the 'games' table

if conn:
    result_df = allintwo.query_database_to_dataframe(conn, query)

conn.close()

In [ ]:
### CHECK TABLES

cursor = conn.cursor()

cursor.execute("""
        SELECT tablename
        FROM pg_tables
        WHERE schemaname = 'public';
    """)


cursor.fetchall()

In [ ]:
### DROP TABLES
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS playerids;")
conn.commit()